In [1]:
!pip3 install Crypto


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 kB 3.5 MB/s eta 0:00:00


In [2]:
!pip3 install pycrypto

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.2/446.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp310-cp310-linux_x86_64.whl size=498546 sha256=8ff48c541acd023d71415fa01a6dfb5c0dad1b43033e255d4ff21d3e38d8c49a
  Stored in directory: /root/.cache/pip/wheels/e8/4b/5b/b10a6fc885057b6ff9fbd5691d7e700d0a9408f80b7e6f12e0
Successfully built pycrypto


In [3]:
!pip install pycryptodome


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.2 MB/s eta 0:00:00


In [12]:
from Crypto.PublicKey import RSA
from Crypto import Random
from Crypto.Hash import SHA
from Crypto.Signature import pkcs1_15


In [13]:
def sign_transaction(self):
    private_key = self.sender._private_key
    h = SHA.new(str(self.to_dict()).encode('utf8'))
    signature = private_key.sign(h, '')
    return binascii.hexlify(signature).decode('ascii')


In [21]:
import hashlib
import binascii
import datetime
import collections
from Crypto.PublicKey import RSA
from Crypto import Random
from Crypto.Hash import SHA
from Crypto.Signature import PKCS1_v1_5

class Client:
    def __init__(self):
        rand_func = Random.new().read
        self._private_key = RSA.generate(1024, rand_func)
        self._public_key = self._private_key.publickey()
        self._signer = PKCS1_v1_5.new(self._private_key)

    @property
    def identity(self):
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

class Transaction:
    def __init__(self, sender, recipient, value):
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = datetime.datetime.now()

    def to_dict(self):
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity
        return collections.OrderedDict({
            'sender': identity,
            'recipient': self.recipient,
            'value': self.value,
            'time': self.time})

    def sign_transaction(self):
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')

def display_transaction(transaction):
    # for transaction in transactions:
    dict = transaction.to_dict()
    print("sender: " + dict['sender'])
    print('----- ')
    print("recipient: " + dict['recipient'])
    print('----- ')
    print("value: " + str(dict['value']))
    print('----- ')
    print("time: " + str(dict['time']))
    print('----- ')

transactions = []
A = Client()
B = Client()
t1 = Transaction(
    A,
    B.identity,
    15.0
)
signature = t1.sign_transaction()
display_transaction(t1)
print("Signature:", signature)


sender: 30819f300d06092a864886f70d010101050003818d0030818902818100f3b313d3cb0318e875e89df32da972d3b969e5f3483d413bcc37459309b6bd8a2b750af09102ccce6b598f3e00847e9ecbdae944d1367de75715705bb6ad69b175b4f2efc00a1681ad74b6f664fa85aad1965f73801ad50a15437fa34db8e097c6d670ee07a2b64f5530acf6c5b6ccd1399087f8d8e783ee7ba78517bb2ddb950203010001
----- 
recipient: 30819f300d06092a864886f70d010101050003818d00308189028181009b3dcd8579993b932573b34390f0c6187477a1ee358d1c30df62a9ebc5a7b61ff1e716285b0b8cd4f9ef2821e0ddbec0cc0f7423828d9c9718f84280f4aa93a3fbad8453e6d32fa25862366b46fba123c7d04fc9a81fc5674ad6b0d62f9d2239184498c7daafd41f79737ee1096feda3183d0af122b3b83a72ee1fcb12f83e0f0203010001
----- 
value: 15.0
----- 
time: 2024-04-16 09:04:28.531189
----- 
Signature: 30da257f3871155e9ea11e170ddc92587098b527e1958740040780aa28789259db270d51e0466d7737c6acdf9137aef53f9b9e915e62d80b29e7fab84fc482c4be85d3cd4592408c7168886fcbd6d61c141de148e3986ab0056f0cfffb731a2191498eeffa9009ea3c3eb5926a9fa2952303e43b0211a79632f3708